# Dynophore notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from dynophores import Dynophore
from dynophores import plot, view3d

In [3]:
dyno_path = Path("../../dynophores/tests/data/1KE7-1/DynophoreApp")
pdb_path = Path("../../dynophores/tests/data/1KE7-1/startframe.pdb")
dcd_path = Path("../../dynophores/tests/data/1KE7-1/trajectory.dcd")

## 3D view

In [4]:
pml_path = dyno_path / "dynophore.pml"
view = view3d.show_dynophore3d(pml_path, pdb_path, dcd_path)
view.display(gui=True, style="ngl")

ThemeManager()

NGLWidget(gui_style='ngl', max_frame=100)

In [5]:
view.render_image(trim=True, factor=2, transparent=True);

In [6]:
view._display_image()

## Statistics

### Load data as `Dynophore` object

In [7]:
dynophore = Dynophore.from_files(dyno_path)

### Plot interactions overview (heatmap)

In [8]:
plot.interactive.superfeatures_vs_envpartners(dynophore);

interactive(children=(SelectMultiple(description='Superfeature name(s):', index=(0,), options=('all', 'AR[4605…

### Plot superfeature occurrences (time series)

In [9]:
plot.interactive.superfeatures_occurrences(dynophore);

interactive(children=(SelectMultiple(description='Superfeature name(s):', index=(0,), options=('all', 'AR[4605…

### Plot interactions for example superfeature (time series)

#### Interaction occurrence

In [10]:
plot.interactive.envpartners_occurrences(dynophore);

interactive(children=(SelectMultiple(description='Superfeature name(s):', index=(0,), options=('AR[4605,4607,4…

#### Interaction distances

In [11]:
plot.interactive.envpartners_distances(dynophore);

interactive(children=(SelectMultiple(description='Superfeature name(s):', index=(0,), options=('AR[4605,4607,4…

#### Interaction profile (all-in-one)

In [12]:
plot.interactive.envpartners_all_in_one(dynophore);

interactive(children=(Select(description='Superfeature name(s):', options=('AR[4605,4607,4603,4606,4604]', 'AR…